# Phase 6: Mesh Reconstruction (Poisson + Trim)
## Alaca Cesmesi Scan-to-HBIM V6 Pipeline

Creates watertight meshes from classified point clouds using Poisson Surface Reconstruction.

**Mesh Selection (from v6 analysis):**
- zemin: trim 10%
- seki: trim 5%
- ana_cephe: no trim (original high quality)
- kemer: no trim (original high quality)
- sacak: trim 10%

**Input:** `gs://alaca-cesme-hbim-v6/processed/v{N}/05_classification/`  
**Output:** `gs://alaca-cesme-hbim-v6/processed/v{N}/06_mesh/`

In [ ]:
!pip install -q open3d google-cloud-storage numpy

In [ ]:
import open3d as o3d
import numpy as np
import json
import os
import time
from datetime import datetime
from google.cloud import storage
from google.colab import auth

auth.authenticate_user()

# Configuration
BUCKET_NAME = "alaca-cesme-hbim-v6"
PROJECT_ID = "concrete-racer-470219-h8"
VERSION = "v1"

# Mesh parameters per element (from v6 analysis)
ELEMENTS = {
    "zemin": {"depth": 10, "trim_pct": 10, "scale": 1.1},
    "seki": {"depth": 10, "trim_pct": 5, "scale": 1.1},
    "ana_cephe": {"depth": 10, "trim_pct": 0, "scale": 1.1},
    "kemer": {"depth": 10, "trim_pct": 0, "scale": 1.1},
    "sacak": {"depth": 10, "trim_pct": 10, "scale": 1.1}
}

# Paths
INPUT_BASE = f"processed/{VERSION}/05_classification/"
OUTPUT_BASE = f"processed/{VERSION}/06_mesh/"

In [ ]:
# GCS functions
def download_from_gcs(bucket_name, blob_name, local_path):
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(local_path)
    print(f"Downloaded: {blob_name}")
    return local_path

def upload_to_gcs(bucket_name, local_path, blob_name):
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(local_path)
    print(f"Uploaded: {blob_name}")
    return f"gs://{bucket_name}/{blob_name}"

In [ ]:
def create_poisson_mesh(pcd, depth=10, scale=1.1):
    """
    Create Poisson surface reconstruction from point cloud.
    
    Args:
        pcd: Point cloud with normals
        depth: Octree depth (higher = more detail)
        scale: Scale factor for bounding cube
    
    Returns:
        mesh, densities
    """
    # Ensure normals exist
    if not pcd.has_normals():
        pcd.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30)
        )
        pcd.orient_normals_consistent_tangent_plane(k=15)
    
    # Poisson reconstruction
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=depth, scale=scale, linear_fit=False
    )
    
    return mesh, np.asarray(densities)


def trim_mesh(mesh, densities, trim_pct):
    """
    Remove low-density vertices (trim the mesh).
    
    Args:
        mesh: Input mesh
        densities: Density values per vertex
        trim_pct: Percentile threshold (0-100)
    
    Returns:
        Trimmed mesh
    """
    if trim_pct <= 0:
        return mesh
    
    threshold = np.percentile(densities, trim_pct)
    vertices_to_remove = densities < threshold
    mesh.remove_vertices_by_mask(vertices_to_remove)
    return mesh


def fix_mesh_normals(mesh):
    """
    Fix mesh normals to point outward from center.
    """
    center = mesh.get_center()
    triangles = np.asarray(mesh.triangles).copy()
    vertices = np.asarray(mesh.vertices)
    
    fixed = 0
    for i, tri in enumerate(triangles):
        v0, v1, v2 = vertices[tri[0]], vertices[tri[1]], vertices[tri[2]]
        face_center = (v0 + v1 + v2) / 3
        normal = np.cross(v1 - v0, v2 - v0)
        outward = face_center - center
        
        if np.dot(normal, outward) < 0:
            triangles[i] = [tri[0], tri[2], tri[1]]
            fixed += 1
    
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    mesh.compute_vertex_normals()
    
    return mesh, fixed

In [ ]:
start_time = time.time()
mesh_stats = {}

for element_name, params in ELEMENTS.items():
    print(f"\n{'='*60}")
    print(f"Processing: {element_name.upper()}")
    print(f"{'='*60}")
    
    # Download point cloud
    local_input = f"/content/{element_name}.ply"
    try:
        download_from_gcs(BUCKET_NAME, f"{INPUT_BASE}{element_name}.ply", local_input)
    except Exception as e:
        print(f"  Skipping {element_name}: {e}")
        continue
    
    pcd = o3d.io.read_point_cloud(local_input)
    n_points = len(pcd.points)
    print(f"  Loaded: {n_points:,} points")
    
    # Create Poisson mesh
    print(f"  Creating Poisson mesh (depth={params['depth']})...")
    mesh, densities = create_poisson_mesh(pcd, depth=params['depth'], scale=params['scale'])
    print(f"  Raw mesh: {len(mesh.vertices):,} vertices, {len(mesh.triangles):,} triangles")
    
    # Trim if needed
    if params['trim_pct'] > 0:
        print(f"  Trimming (removing bottom {params['trim_pct']}% density)...")
        mesh = trim_mesh(mesh, densities, params['trim_pct'])
        print(f"  After trim: {len(mesh.vertices):,} vertices, {len(mesh.triangles):,} triangles")
    
    # Fix normals
    print("  Fixing normals...")
    mesh, fixed_count = fix_mesh_normals(mesh)
    print(f"  Fixed: {fixed_count:,} triangles ({100*fixed_count/len(mesh.triangles):.1f}%)")
    
    # Clean mesh
    mesh.remove_degenerate_triangles()
    mesh.remove_duplicated_triangles()
    mesh.remove_duplicated_vertices()
    mesh.remove_non_manifold_edges()
    
    # Save
    local_output = f"/content/{element_name}_mesh.ply"
    o3d.io.write_triangle_mesh(local_output, mesh)
    upload_to_gcs(BUCKET_NAME, local_output, f"{OUTPUT_BASE}{element_name}_poisson.ply")
    
    # Stats
    mesh_stats[element_name] = {
        "input_points": n_points,
        "vertices": len(mesh.vertices),
        "triangles": len(mesh.triangles),
        "depth": params['depth'],
        "trim_pct": params['trim_pct'],
        "normals_fixed": fixed_count
    }

elapsed_time = time.time() - start_time
print(f"\nTotal processing time: {elapsed_time:.1f} seconds")

In [ ]:
# Summary
print("\n" + "="*60)
print("MESH RECONSTRUCTION SUMMARY")
print("="*60)
print(f"\n{'Element':<12} {'Points':>10} {'Vertices':>10} {'Triangles':>10} {'Trim':>6}")
print("-"*50)
for name, stats in mesh_stats.items():
    print(f"{name:<12} {stats['input_points']:>10,} {stats['vertices']:>10,} {stats['triangles']:>10,} {stats['trim_pct']:>5}%")

# Save stats
final_stats = {
    "phase": "06_mesh",
    "meshes": mesh_stats,
    "processing_time_sec": elapsed_time,
    "timestamp": datetime.now().isoformat(),
    "pipeline_version": "v6"
}

local_stats = "/content/06_mesh_stats.json"
with open(local_stats, 'w') as f:
    json.dump(final_stats, f, indent=2)
upload_to_gcs(BUCKET_NAME, local_stats, f"{OUTPUT_BASE}06_mesh_stats.json")

In [ ]:
# Status for n8n
status = {
    "phase": "06_mesh",
    "status": "success",
    "version": VERSION,
    "outputs": {
        "meshes": [f"gs://{BUCKET_NAME}/{OUTPUT_BASE}{name}_poisson.ply" for name in mesh_stats.keys()],
        "stats": f"gs://{BUCKET_NAME}/{OUTPUT_BASE}06_mesh_stats.json"
    },
    "metrics": {
        "n_meshes": len(mesh_stats),
        "total_triangles": sum(s['triangles'] for s in mesh_stats.values()),
        "processing_time": f"{elapsed_time:.1f}s"
    },
    "timestamp": datetime.now().isoformat(),
    "next_phase": "07_ifc"
}

print("\n" + "="*60)
print("PHASE 6 COMPLETE")
print("="*60)
print(json.dumps(status, indent=2))